In [ ]:
hist_subtypes = pd.read_csv('../heatmap_header.txt', sep='\t', index_col=0)
hist_subtypes = hist_subtypes.drop('TP53 mut', axis=0)
hist_colors = {'Serous': '#92C45A', 
               'Endometrioid': '#ED8CBC', 
               'CNV_high': '#EF803D',
               'POLE':'#D9432E', 
               'MSI-H':'#5AA352', 
               'CNV_low': '#3A7AB6', 
               'False': '#D9D9D9', 
               'True': '#616161'}
handles = [mpatches.Patch(color=color, label=group) for group, color in hist_colors.iteritems()]
reorder = [6, 2, 3, 1, 0, 5, 4, 7]
handles = [handles[ind] for ind in reorder]
cmap = sns.diverging_palette(220, 10, as_cmap=True)
cmap.set_bad('#BDBDBD')
cmap.set_over('#9E031A')
cmap.set_under('#0C4A60')

In [ ]:
hist_colors = {'Serous': '#92C45A', 
               'Endometrioid': '#ED8CBC', 
               'CNV_high': '#EF803D',
               'POLE':'#D9432E', 
               'MSI-H':'#5AA352', 
               'CNV_low': '#3A7AB6', 
               'False': '#D9D9D9', 
               'True': '#616161', 
               'nan':'#FFFFFF',
               'IA':'#FF5E34',
               'IB':'#FF5E34',
               'II':'#C9463D', 
               'IIIA':'#962E40',
               'IIIB':'#962E40',
               'IIIC1':'#962E40',
               'IIIC2':'#962E40',
               'IVB':'#5E1742',
               'FIGO grade 1':'#3BFDE1',
               'FIGO grade 2':'#46AFE8',
               'FIGO grade 3':'#3158FF'}

In [ ]:
genes_to_plot = commonFns.fileToList('../outliers_groups/fdr_0.01/phospho/DDR_endo_druggable_up_high.outlier_sites_in_high.txt')
genes_to_highlight = commonFns.fileToList('../outliers_groups/fdr_0.05/phospho/DDR_serous_druggable_up_high.outlier_sites_in_high.txt')
order = commonFns.fileToList('../NHEJ/DDR_order.txt')
title = 'Druggable genes significantly differential in endometrioid DDR-high samples'

df = pd.read_csv('../outliers/UCEC_phospho_outliers_both.fraction_outliers.txt', sep='\t').set_index('geneSymbol')

heatmap_table = df.loc[genes_to_plot, order].sort_index()

fig = plt.figure(figsize=(0.13*len(list(hist_subtypes)), (0.25*(len(heatmap_table)))))
gs = matplotlib.gridspec.GridSpec(4, 2, 
                                  width_ratios=[len(list(hist_subtypes)), 2], 
                                  height_ratios=[len(hist_subtypes)]+[len(heatmap_table)/3 for i in range(0,3)], 
                                  left=0.20, top=0.95, bottom=0.05, right=0.82, wspace=0.05, hspace=0.005, 
                                  )

catax = plt.subplot(gs[0, 0])
hist_subtypes = hist_subtypes.reindex(heatmap_table.columns, axis=1)
cat = catheat.heatmap(hist_subtypes, cmap=hist_colors, ax=catax, 
                      xticklabels=False, yticklabels=hist_subtypes.index, legend=False)

catax.set_yticklabels(catax.get_yticklabels(), rotation=0)
catax.set_ylabel('')
catax.set_title('%s' %(title), fontsize=16, va='bottom')

heatax = plt.subplot(gs[1:,0])
cbarax = plt.subplot(gs[1,1])
sns.heatmap(heatmap_table, ax=heatax,
            cbar_ax=cbarax, 
            cbar_kws=dict(label='Fraction outliers', shrink=0.3),
            cmap=cmap, 
            xticklabels=False, 
            yticklabels=heatmap_table.index, 
            vmin=-1, 
            vmax=1)
heatax.set_ylabel('')

if genes_to_highlight != None:
    new_labels = []
    for gene in heatax.get_yticklabels():
        gene = gene.get_text()
        if gene in genes_to_highlight:
            new_labels.append(str(gene)+'*')
        else:
            new_labels.append(gene)
    heatax.set_yticklabels(new_labels)

legendax = plt.subplot(gs[3,1])
legendax.axis('off')
legendax.legend(handles=handles, loc=(0, 0.0),labelspacing=0)

# plt.savefig('%s.pdf' %(title), dpi=500)
plt.show()
plt.close()